In [ ]:
!git clone https://github.com/marco-c/crashcorrelations

In [ ]:
import os
import errno
import json
import gzip

from crashcorrelations import download_data
from crashcorrelations import crash_deviations

In [ ]:
channels = ['release', 'beta', 'aurora', 'nightly']

channel_to_version = {
  'release': '47.0.1',
  'beta': '48.0b',
  'aurora': '49.0a2',
  'nightly': '50.0a1',
}

results = {
  'release': {},
  'beta': {},
  'aurora': {},
  'nightly': {},
}

In [ ]:
download_data.set_token('YOUR_SOCORRO_TOKEN_HERE')

for channel in channels:
    download_data.download_crashes(version=channel_to_version[channel], days=5)

In [ ]:
for channel in channels:
    df_a = crash_deviations.get_crashes(sc, version=channel_to_version[channel], days=5)
    signatures = download_data.get_top_50(channel_to_version[channel], 5)
    for signature in signatures:
        df_b = df_a.filter(df_a['signature'] == signature)
        results[channel][signature] = crash_deviations.find_deviations(sc, df_a, df_b, min_support_diff=0.15, min_corr=0.03, max_addons=50)

In [ ]:
try:
    os.mkdir('output')
except OSError as e:
    if e.errno != errno.EEXIST:
        raise e

with gzip.open('output/top50_results.json.gz', 'wb') as f:
    json.dump(results, f)